In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,TrainingArguments
from peft import LoraConfig
import torch
from datasets import load_dataset

In [ ]:
pip install trl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
file_path = '/content/drive/My Drive/dataset_finetuning/train.csv'
df = pd.read_csv(file_path)
df.head()

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance


In [ ]:
df = df.drop(columns=['id','topic'])
df.head()

,dialogue,summary
0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw..."
1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...
2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...
3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...
4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...


In [ ]:
df.rename(columns = {'dialogue':'input', 'summary':'output'}, inplace = True)
df.head()

,input,output
0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw..."
1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...
2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...
3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...
4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import json
data = [
    {"input_text": row["input"], "target_text": row["output"]}
    for _, row in df.iterrows()
]
json_file_path = "/content/drive/My Drive/dataset_finetuning/train.json"
with open(json_file_path, "w",encoding= "utf-8") as json_file:
    json.dump(data, json_file, indent=4)

print(f"JSON file saved to {json_file_path}")


JSON file saved to /content/drive/My Drive/dataset_finetuning/train.json


In [ ]:
import os

if os.path.exists(json_file_path):
    print("✅ JSON file saved")
else:
    print("❌ JSON file not found")


✅ JSON file saved


In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset
json_file_path = "/content/drive/My Drive/dataset_finetuning/train.json"
dataset = load_dataset("json", data_files=json_file_path)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 12460
    })
})
{'input_text': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'l

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from transformers import AutoTokenizer
from huggingface_hub import notebook_login

# Hugging Face'e giriş yap
notebook_login()

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)  # Güvenli giriş

def preprocess_function(examples):
    # Chat modeline uygun hale getirme
    messages = [
        [{"role": "user", "content": input_text}, {"role": "assistant", "content": target_text}]
        for input_text, target_text in zip(examples["input_text"], examples["target_text"])
    ]

    # Chat template uygula ve tokenize et
    input_texts = [
        tokenizer.apply_chat_template(m, tokenize=True, add_generation_prompt=True) for m in messages
    ]

    # Model için labels tokenizasyonu
    labels = tokenizer(examples["target_text"], max_length=256, truncation=True)

    # Eğer tokenizer'ın dönüş değeri tensor ise, listeye çevir
    if isinstance(input_texts[0], list):
        input_ids = input_texts
    else:
        input_ids = [t.tolist() if hasattr(t, "tolist") else list(t) for t in input_texts]

    return {"input_ids": input_ids, "labels": labels["input_ids"]}

# Dataset'i batch halinde tokenize et
tokenized_dataset = dataset.map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

In [ ]:
print(dataset["train"][0])

{'input_text': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.", 'target_text': "Mr. Smith's getting a c

In [49]:
from transformers import AutoModelForCausalLM
import torch


model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model = AutoModelForCausalLM.from_pretrained(modesl_name, token=True)

raw_input = tokenized_dataset["train"][0]["input_ids"]
input_text = tokenizer.decode(raw_input, skip_special_tokens=True)


formatted_input = f"Can you summarize this conversation briefly?\n\n{input_text}"


input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids

output_tokens = model.generate(input_ids, max_new_tokens=100, do_sample=True, temperature=0.7)
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print("🔹 **Eğitimsiz Modelin Özetlemesi:**")
print(output_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


🔹 **Eğitimsiz Modelin Özetlemesi:**
Can you summarize this conversation briefly?

<｜User｜>Can you summarize this conversation briefly?

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give y

In [ ]:
print(output_text)

Can you summarize this conversation briefly?

<｜User｜>#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.<｜Assistant｜>Mr

In [ ]:
!pip install transformers peft accelerate datasets


In [ ]:
training_args = TrainingArguments(
    output_dir="./deepseek_finetuned",  # Modelin kaydedileceği yer
    per_device_train_batch_size=2,  # Her GPU'da kullanılacak batch boyutu
    num_train_epochs=3,  # Kaç epoch eğiteceğiz?
    save_steps=500,  # Kaç adımda bir modeli kaydedelim?
    save_total_limit=2,  # Kaç tane checkpoint tutulsun?
    logging_steps=50,  # Her 50 adımda bir log kaydı al
    evaluation_strategy="epoch",  # Her epoch sonunda değerlendirme yap
    learning_rate=2e-4,  # Öğrenme oranı
    weight_decay=0.01,  # Ağırlık bozulmasını engellemek için
    bf16=True,  # GPU kullanıyorsan eğitimi hızlandırır
    push_to_hub=False  # Eğer Hugging Face Hub'a kaydetmek istersen True yap
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from datasets import load_dataset

# Eğer Hugging Face'e dataset'i yüklediysen, buradan çağırabilirsin
# dataset = load_dataset("bilge/aya-finetuning-dataset")

# Train/Test ayırımı yapalım (%80 train, %20 test)
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.2)

# Ayırdığımız datasetleri görelim
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"✅ Train Set Boyutu: {len(train_dataset)}")
print(f"✅ Test Set Boyutu: {len(test_dataset)}")


✅ Train Set Boyutu: 9968
✅ Test Set Boyutu: 2492


In [ ]:
def preprocess_function(examples):
    # Prompt'u input_text'e ekleyelim
    formatted_inputs = [f"Can you summarize this conversation briefly?\n\n{text}" for text in examples["input_text"]]

    # Chat formatına uygun hale getirme
    messages = [
        [{"role": "user", "content": input_text}, {"role": "assistant", "content": target_text}]
        for input_text, target_text in zip(formatted_inputs, examples["target_text"])
    ]

    # Chat template uygula ve tokenize et
    input_texts = [tokenizer.apply_chat_template(m, tokenize=False, add_generation_prompt=True) for m in messages]

    # Model için labels tokenizasyonu (LoRA sonrası bozulmaması için `return_tensors="pt"` kaldırıldı)
    inputs = tokenizer(
        input_texts,
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        examples["target_text"],
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"]
    }

# Dataset'i tekrar tokenize edelim
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

In [ ]:
print(len(tokenized_dataset["train"][0]["input_ids"]))  # Uzunluk hep 256 olmalı
print(len(tokenized_dataset["train"][0]["labels"]))  # Uzunluk hep 256 olmalı


256
256


In [ ]:
import pickle

with open("tokenized_dataset.pkl", "wb") as f:
    pickle.dump(tokenized_dataset, f)


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# LoRA yapılandırmasını oluştur
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

# Modeli yükleyelim
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model = AutoModelForCausalLM.from_pretrained(model_name, token=True)

# LoRA adaptörünü ekleyelim
model = get_peft_model(model, lora_config)

# Modeli eğitime hazır hale getir
model = prepare_model_for_kbit_training(model)

# **TÜM PARAMETRELERİ EĞİTİME AÇ**
for param in model.parameters():
    param.requires_grad = True
model.gradient_checkpointing_enable()

# GPU kontrolü
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


PeftModel(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=1536, o

In [ ]:
!nvidia-smi


Mon Mar 17 08:14:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             50W /  400W |    7309MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
print(len(train_dataset[0]["input_ids"]))  # 256 olmalı
print(len(train_dataset[0]["labels"]))  # 256 olmalı


207
34


In [ ]:
with open("tokenized_dataset.pkl", "rb") as f:
    tokenized_dataset = pickle.load(f)

# **Train ve Test Dataset'ini Tekrar Tanımla!**
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]


In [ ]:
print(len(train_dataset[0]["input_ids"]))  # 256 olmalı
print(len(train_dataset[0]["labels"]))  # 256 olmalı


256
256


In [ ]:
print(model)

PeftModel(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=1536, o

In [ ]:
training_args = TrainingArguments(
    output_dir="./aya_finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)


In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


base_model.model.model.embed_tokens.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.k_proj.weight
base_model.model.model.layers.0.self_attn.k_proj.bias
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.v_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.o_proj.weight
base_model.model.model.layers.0.mlp.gate_proj.weight
base_model.model.model.layers.0.mlp.up_proj.weight
base_model.model.model.layers.0.mlp.down_proj.weight
base_model.model.model.layers.0.input_layernorm.weight
base_model.model.model.layers.0

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Toplam Parametre: {total_params / 1e9:.2f}B")  # Milyar cinsinden göster
print(f"Eğitilebilir Parametre: {trainable_params / 1e6:.2f}M")  # Milyon cinsinden göster


Toplam Parametre: 1.78B
Eğitilebilir Parametre: 1778.18M


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.313600,No log
2,0.286500,No log
3,0.142100,No log


TrainOutput(global_step=7476, training_loss=0.2717528716965497, metrics={'train_runtime': 3753.3509, 'train_samples_per_second': 7.967, 'train_steps_per_second': 1.992, 'total_flos': 7.095677035216896e+16, 'train_loss': 0.2717528716965497, 'epoch': 3.0})

In [ ]:
model.save_pretrained("trained_model")
tokenizer.save_pretrained("trained_model")


('trained_model/tokenizer_config.json',
 'trained_model/special_tokens_map.json',
 'trained_model/tokenizer.json')

In [ ]:
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=1536, o

In [ ]:
print(dataset)
print(dataset['train'][10])

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 12460
    })
})
{'input_text': "#Person1#: Could you do me a favor?\n#Person2#: Sure. What is it?\n#Person1#: Could you run over to the store? We need a few things.\n#Person2#: All right. What do you want me to get?\n#Person1#: Well, could you pick up some sugar?\n#Person2#: Okay. How much?\n#Person1#: A small bag. I guess we also need a few oranges.\n#Person2#: How many?\n#Person1#: Oh, let's see. . . About six.\n#Person2#: Anything else?\n#Person1#: Yes. We're out of milk.\n#Person2#: Okay. How much do you want me to get? A gallon?\n#Person1#: No. I think a half gallon will be enough.\n#Person2#: Is that all?\n#Person1#: I think so. Have you got all that?\n#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of milk.\n#Person1#: Do you have enough money?\n#Person2#: I think so.\n#Person1#: Thanks very much. I appreciate it.", 'target_text': '#Person1# asks #Pers

In [51]:

test_results = []

for i, test_sample in enumerate(test_dataset):
    input_text = tokenizer.decode(test_sample["input_ids"], skip_special_tokens=True)

    formatted_input = f"Can you summarize this conversation briefly?\n\n{input_text}"


    input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids.to(model.device)

    with torch.no_grad():
        output_tokens = model.generate(input_ids, max_new_tokens=100, do_sample=True, temperature=0.7)

    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


    test_results.append({"input": input_text, "output": output_text})

    print(f"🔹 **Test {i+1}:**")
    print("🔹 **Input:**", input_text)
    print("🔹 **Model Output:**", output_text)
    print("=" * 50)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


🔹 **Test 1:**
🔹 **Input:** <｜User｜>Can you summarize this conversation briefly?

#Person1#: What would you say if I told you I was going to quit school?
#Person2#: I'd say, think twice about it. Well, you are not going to quit school, are you?
#Person1#: I don't know. I failed my exam.
#Person2#: What did you get?
#Person1#: A B plus.
#Person2#: That's not bad.
#Person1#: But I should have aced it. I mean I finished the paper so quickly, thinking it's a piece of cake. But when I knew my result, I was like what? That can't be true!
#Person2#: Listen, John. I understand that you are such an excellent student and I know you must have lots of stress, but you really want to give up?
#Person1#: No, I don't. I just don't know how to handle this. This is my first time, you know.
#Person2#: I know. Don't worry. Do you know what you should do?
#Person1#: What?
#Person2#: You need to blow off some steam. The world is not on your shoulder.
#Person1#: How
🔹 **Model Output:** Can you summarize this 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


🔹 **Test 2:**
🔹 **Input:** <｜User｜>Can you summarize this conversation briefly?

#Person1#: What is the difference between a lesson and a lecture?
#Person2#: Well, they are both ways of imparting knowledge, but the main difference is that you participate in a lesson whereas you just listen to a lecture. A lecture is generally given to a much larger group.<｜Assistant｜>#Person2# tells #Person1# the difference between a lesson and a lecture.<｜Assistant｜><think>

🔹 **Model Output:** Can you summarize this conversation briefly?

<｜User｜>Can you summarize this conversation briefly?

#Person1#: What is the difference between a lesson and a lecture?
#Person2#: Well, they are both ways of imparting knowledge, but the main difference is that you participate in a lesson whereas you just listen to a lecture. A lecture is generally given to a much larger group.<｜Assistant｜>#Person2# tells #Person1# the difference between a lesson and a lecture.<｜Assistant｜><think>
Okay, so I need to summarize this 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


🔹 **Test 40:**
🔹 **Input:** <｜User｜>Can you summarize this conversation briefly?

#Person1#: Hi, Joe! Many people in our department are leaving for Thanksgiving. What are you going to do?
#Person2#: I think I'll just stay at home. I may see some friends and watch a few DVDs. And I probably would go to the Ottawa Car Show. What about you, Ariel? What's your plan?
#Person1#: I am going to do some traveling with my family.
#Person2#: Oh, where to?
#Person1#: Toronto. And on our way, we're going to visit my aunt, Valley, in Kingston.
#Person2#: That's exciting. How long do you plan to stay?
#Person1#: Two days. I have a get-together party with my college friends on the tenth in Toronto. Oh, I'm afraid I've got to go now. My husband is waiting for me. And we'll have to pick up Daniel from school. Have a nice holiday, Joe!
#Person2#: Thank you. You too!<｜Assistant｜>Ariel asks Joe about his plan for Thanksgiving. Joe says he will stay at home and may go to a car show. Ariel will travel with h

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


🔹 **Test 41:**
🔹 **Input:** <｜User｜>Can you summarize this conversation briefly?

#Person1#: Robbie, this new walkman is really wonderful.
#Person2#: Richard and Marilyn bought it for me for my birthday.
#Person1#: They are so kind. You are very lucky, Robbie, to have such a nice family.
#Person2#: Is something wrong, Alexandra?
#Person1#: No, nothing.
#Person2#: Yes, there is. I can tell. What's the matter? Come on, you can tell me. What's up?
#Person1#: I received a letter from my parents this morning. I miss them very much. The Molinas treat me so nicely and I like being with your family so much...but when I received the letter with photographs of my family, I cried.
#Person2#: You really miss your family,don't you?
#Person1#: Yes, I know I must seem silly. It's not like I have nobody. I like the Molinas very much, and they're so kind to me.
#Person2#: Hey, why don't we go out for a hamburger and French fries? That'll cheer you up. And you can use my walkman.
#Person1#: That's a goo

KeyboardInterrupt: 

In [56]:
# prompt: save LoRA  adaptors for downloading

import shutil

# Save the LoRA adapter
model.save_pretrained("lora_adapter")

# Create a zip archive of the adapter
shutil.make_archive("lora_adapter", 'zip', "lora_adapter")

# Download the zip file (you can replace this with your preferred download method)
from google.colab import files
files.download("lora_adapter.zip")


KeyboardInterrupt: 

In [57]:
from peft import PeftModel

# Eğitilmiş modeli yükle
model_path = "trained_model"  # Kaydettiğin modelin yolu

# Eğer model gerçekten LoRA ile eğitildiyse, PeftModel olarak yüklenmeli
model = PeftModel.from_pretrained(model_path)


TypeError: PeftModel.from_pretrained() missing 1 required positional argument: 'model_id'

In [61]:
import shutil

# trained_model içinden yalnızca LoRA dosyalarını al
source_dir = "trained_model"
target_dir = "lora_adapter"

# Eğer daha önce varsa, silip temizle
shutil.rmtree(target_dir, ignore_errors=True)

# Sadece LoRA ile ilgili dosyaları yeni klasöre kopyala
shutil.copytree(source_dir, target_dir, ignore=shutil.ignore_patterns("pytorch_model.bin", "config.json"))

print("✅ Sadece LoRA adaptörü başarıyla kopyalandı!")


✅ Sadece LoRA adaptörü başarıyla kopyalandı!


In [62]:
ls lora_adapter/

adapter_config.json        README.md                tokenizer_config.json
adapter_model.safetensors  special_tokens_map.json  tokenizer.json


In [66]:
!zip -r lora_adapter.zip lora_adapter

  adding: lora_adapter/ (stored 0%)
  adding: lora_adapter/adapter_config.json (deflated 53%)
  adding: lora_adapter/tokenizer_config.json (deflated 85%)
  adding: lora_adapter/README.md (deflated 66%)
  adding: lora_adapter/adapter_model.safetensors (deflated 8%)
  adding: lora_adapter/special_tokens_map.json (deflated 73%)
  adding: lora_adapter/tokenizer.json (deflated 81%)


In [67]:
from google.colab import files
files.download("lora_adapter.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
ls -l /content/


total 6072
drwxr-xr-x  5 root root    4096 Mar 17 09:44 aya_finetuned/
drwx------  6 root root    4096 Mar 17 05:16 drive/
drwxr-xr-x  2 root root    4096 Mar 17 10:35 lora_adapter/
-rw-r--r--  1 root root 6188322 Mar 17 11:42 lora_adapter.zip
drwxr-xr-x  1 root root    4096 Mar 13 13:31 sample_data/
-rw-r--r--  1 root root    2112 Mar 17 08:14 tokenized_dataset.pkl
drwxr-xr-x  2 root root    4096 Mar 17 10:35 trained_model/
drwxr-xr-x 10 root root    4096 Mar 17 08:15 wandb/
